In [146]:
# Question 1

from collections import deque

def findSafeResidents(grid):
    if not grid:
        return []

    num_rows, num_cols = len(grid), len(grid[0])
    movements = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    safe_residents = []

    def is_valid(x, y):
        return 0 <= x < num_rows and 0 <= y < num_cols

    q = deque([(i, j) for i in range(num_rows) for j in range(num_cols) if grid[i][j] == 2])
    while q:
        x, y = q.popleft()
        for dx, dy in movements:
            nx, ny = x + dx, y + dy
            if is_valid(nx, ny) and grid[nx][ny] == 1:
                grid[nx][ny] = 2  
                q.append((nx, ny))

    for i in range(num_rows):
        for j in range(num_cols):
            if grid[i][j] == 1:
                safe_residents.append([i, j])

    return safe_residents


In [147]:
# Question 2

import heapq
import math

def reverse_roads(n, roads):
    graph = {i: [] for i in range(1, n+1)}
    for road in roads:
        graph[road[0]].append((road[1], 0))  
        graph[road[1]].append((road[0], 1)) 
    
    min_distance = [math.inf] * (n + 1)
    min_distance[1] = 0
    priority_queue = [(0, 1)]  
    
    while priority_queue:
        distance, city = heapq.heappop(priority_queue)
        if distance > min_distance[city]:  
            continue
        for neighbor, road_cost in graph[city]:
            new_distance = distance + road_cost
            if new_distance < min_distance[neighbor]:
                min_distance[neighbor] = new_distance
                heapq.heappush(priority_queue, (new_distance, neighbor))
    
    return min_distance[n] if min_distance[n] != math.inf else -1 


In [148]:
# Question 3

from collections import defaultdict, deque

def findAllRecipes(recipes, ingredients, supplies):
    recipe_graph = defaultdict(list)
    recipe_indegree = defaultdict(int)
    for i, recipe in enumerate(recipes):
        for ingredient in ingredients[i]:
            recipe_graph[ingredient].append(recipe)
            recipe_indegree[recipe] += 1
    
    supply_queue = deque(supplies)
    created_recipe_set = set()
    
    while supply_queue:
        ingredient = supply_queue.popleft()
        if ingredient in recipe_graph:
            for recipe in recipe_graph[ingredient]:
                recipe_indegree[recipe] -= 1
                if recipe_indegree[recipe] == 0:
                    created_recipe_set.add(recipe)
                    supply_queue.append(recipe)
    
    return list(created_recipe_set)

In [149]:
# Question 4

class UnionFind4:
    def __init__(self, size):
        self.parent = list(range(size))
        self.rank = [0] * size
    
    def find(self, element):
        if self.parent[element] != element:
            self.parent[element] = self.find(self.parent[element])
        return self.parent[element]
    
    def union(self, element1, element2):
        root1 = self.find(element1)
        root2 = self.find(element2)
        if root1 != root2:
            if self.rank[root1] < self.rank[root2]:
                self.parent[root1] = root2
            elif self.rank[root1] > self.rank[root2]:
                self.parent[root2] = root1
            else:
                self.parent[root2] = root1
                self.rank[root1] += 1
                
def connectInMinCost(connections):
    connections.sort(key=lambda x: x[2])
    
    num_nodes = max(max(connection[:2]) for connection in connections) + 1
    uf = UnionFind4(num_nodes)
    min_cost = 0
    
    for connection in connections:
        src, dst, cst = connection
        if uf.find(src) != uf.find(dst):
            uf.union(src, dst)
            min_cost += cst
    
    return min_cost

In [150]:
# Question 5

def busRouter(num_routes, dependencies):
    route_graph = [[] for _ in range(num_routes)]
    for dependency in dependencies:
        route_graph[dependency[1]].append(dependency[0])

    def depth_first_search(node, visit_status):
        if visit_status[node] == 1:
            return True
        visit_status[node] = 1
        for neighbor in route_graph[node]:
            if depth_first_search(neighbor, visit_status):
                return True
        visit_status[node] = 2
        return False

    for i in range(num_routes):
        visited = [0] * num_routes
        if depth_first_search(i, visited):
            return False

    return True


In [143]:
# Question 6

from copy import deepcopy

def gameOfThrones(n: int, friendships: list[int], queries: list[int]) -> list[int]:
    
    def new_graph(n):
        graph = {}
        for i in range(1, n+1):
            graph[i] = []
        return graph
    
    def initialize_graph(friendships, graph = {}):    
        for edge in friendships:
            vertex1, vertex2 = edge
            graph[vertex1].append(vertex2)
            graph[vertex2].append(vertex1)
        return graph
    
    def delete_friendship(breakup, graph):
        vertex1, vertex2 = breakup
        if vertex1 in graph:
            if vertex2 in graph[vertex1]:
                graph[vertex1].remove(vertex2)
        if vertex2 in graph:
            if vertex1 in graph[vertex2]:
                graph[vertex2].remove(vertex1)
        return graph
        
    def getRidOfTheWeakOnes(temp_graph):
        def query_3(temp_graph):
            for v in temp_graph:
                if not temp_graph[v]:
                    pass
                elif v < min(temp_graph[v]) :
                    del temp_graph[v]
                    for j in temp_graph:
                        if v in temp_graph[j]:
                            temp_graph[j].remove(v)
                    new_graph = query_3(temp_graph)
                    return new_graph
            return temp_graph

        new_graph = query_3(temp_graph)    
        return len(new_graph.keys())
            
    remaining_nobles = []
    graph = new_graph(n)
    graph = initialize_graph(friendships, graph)
    for i in queries:
        if i[0] == 1:
            graph = initialize_graph([[i[1], i[2]]], graph)
        elif i[0] == 2:
            graph = delete_friendship([i[1], i[2]], graph)
        else:
            graph_copy = deepcopy(graph)
            remaining_nobles.append(getRidOfTheWeakOnes(graph_copy))
            
    return remaining_nobles

In [151]:
# Question 7

import heapq

def optimalSignal(times: list[list[int]], num_nodes: int, source_node: int) -> int:
    node_graph = [[] for _ in range(num_nodes + 1)]  
    for x, y, z in times:
        node_graph[x].append((y, z))
    
    distances = [float('inf')] * (num_nodes + 1)
    distances[source_node] = 0  
    
    priority_queue = [(0, source_node)]  
    
    while priority_queue:
        dist, node = heapq.heappop(priority_queue)
        if dist > distances[node]:
            continue
        for neighbor, weight in node_graph[node]:
            if distances[node] + weight < distances[neighbor]:
                distances[neighbor] = distances[node] + weight
                heapq.heappush(priority_queue, (distances[neighbor], neighbor))
    
    if float('inf') in distances[1:]:
        return -1
    
    return max(distances[1:])

In [152]:
# Question 8

class UnionFind8:
    def __init__(self, size):
        self.groups = list(range(size))
        self.height = [0] * size
    
    def find(self, element):
        if self.groups[element] != element:
            self.groups[element] = self.find(self.groups[element])
        return self.groups[element]
    
    def union(self, element1, element2):
        root1 = self.find(element1)
        root2 = self.find(element2)
        if root1 != root2:
            if self.height[root1] < self.height[root2]:
                self.groups[root1] = root2
            elif self.height[root1] > self.height[root2]:
                self.groups[root2] = root1
            else:
                self.groups[root2] = root1
                self.height[root1] += 1

def manhattan_distance(pt1, pt2):
    return abs(pt1[0] - pt2[0]) + abs(pt1[1] - pt2[1])

def connectAllPoints(points: list[list[int]]) -> int:
    num_points = len(points)
    all_edges = []

    for i in range(num_points):
        for j in range(i + 1, num_points):
            dist = manhattan_distance(points[i], points[j])
            all_edges.append((dist, i, j))
    
    all_edges.sort()

    mst_edges = []
    uf = UnionFind8(num_points)

    for dist, p1, p2 in all_edges:
        if uf.find(p1) != uf.find(p2):
            mst_edges.append((dist, p1, p2))
            uf.union(p1, p2)

    total_cost = sum(dist for dist, _, _ in mst_edges)
    return total_cost

In [9]:
from queue import Queue 

adj_list = {
    "A":["B", "D"],
    "B":["A", "C"],
    "C":["B"],
    "D":["A", "E", "F"],
    "E":["D", "F", "G"],
    "F":["D", "E", "H"],
    "G":["E", "H"],
    "H":["G", "F"]
}

# BFS Code

visited = {}
level = {} # distance dictionary
parent = {}
bfs_traversal_output = []
queue = Queue()


for node in adj_list.keys():
    visited[node] = False
    parent[node] = None
    level[node] = -1 #inf
    
s = "A"
visited[s] = True
level[s] = 0
queue.put(s)

while not queue.empty():
    u = queue.get()
    bfs_traversal_output.append(u)
    
    for v in adj_list[u]:
        if not visited[v]:
            visited[v] = True
            parent[v] = u
            level[v] = level[u]+1
            queue.put(v)
            
print(bfs_traversal_output)


['A', 'B', 'D', 'C', 'E', 'F', 'G', 'H']


In [10]:
# shortest distance of all nodes from source node
print(level)

{'A': 0, 'B': 1, 'C': 2, 'D': 1, 'E': 2, 'F': 2, 'G': 3, 'H': 3}


In [11]:
# shortest route from v to source node (This will work only in a fully connected graph.
# In a graph that is not fully connected, you will need to work out a condition to handle it)
v = "G"
path = []
while v is not None:
    path.append(v)
    v = parent[v]
    
path.reverse()
print(path)

['A', 'D', 'E', 'G']
